## 13.2 Dagger-Hilt - Podstawy - ROOM

W tej aplikacji zaimplementujemy architekturę **MVVM** z wykorzystaniem `Dagger-Hilt` oraz lokalną bazą `ROOM`. Rozpocznijmy od dodania zależności.

In [ ]:
// build.gradle(Project)
id 'com.google.dagger.hilt.android' version '2.44' apply false

In [ ]:
plugins {
    id 'com.android.application'
    id 'com.google.dagger.hilt.android'
}

dependencies {

    // ROOM
    implementation "androidx.room:room-runtime:2.4.3"
    annotationProcessor "androidx.room:room-compiler:2.4.3"

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"

    implementation "com.google.dagger:hilt-android:2.44"
    annotationProcessor 'androidx.hilt:hilt-compiler:1.0.0'
    annotationProcessor "com.google.dagger:hilt-android-compiler:2.44"
}

Zdefiniujmyn model danych.

In [ ]:
@Entity(tableName = "student")
public class Student {

    @PrimaryKey(autoGenerate = true)
    private int id;

    private final String name;

    public Student(int id, String name) {
        this.id = id;
        this.name = name;
    }

    public String getName() {
        return name;
    }

    public int getId() {
        return id;
    }
}

Następnie dodajmy klasę zwracającą instancję bazy - tutaj musimy przekazać `Context` w metodzie `getDatabase`.

In [ ]:
@Database(entities = {Student.class}, version = 1, exportSchema = false)
public abstract class AppDatabase extends RoomDatabase{

        public abstract AppDao appDao();

        private static volatile AppDatabase INSTANCE;

        private static final int NUMBER_OF_THREADS = 4;
        public static final ExecutorService databaseWriteExecutor =
                Executors.newFixedThreadPool(NUMBER_OF_THREADS);

        public static AppDatabase getDatabase(final Context context) {
            if (INSTANCE == null) {
                synchronized (AppDatabase.class) {
                    if (INSTANCE == null) {
                        INSTANCE = Room.databaseBuilder(context.getApplicationContext(),
                                        AppDatabase.class, "student_database_java")
                                .build();
                    }
                }
            }
            return INSTANCE;
        }
}


Kolejnym elementem bazy jest `Dao` - tutaj dodamy dwie metody do odczytania wszystkich danych oraz dodania pojedynczego elementu.

In [ ]:
@Dao
public interface AppDao {
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void addItem(Student student);

    @Query("SELECT * FROM student")
    LiveData<List<Student>> readAllData();
}

W metodzie dostarczającej musimy przekazać `Application` jako `Context` do utworzenia `ROOM`. Biblioteka `Hilt` automatycznie wykorzysta instancję klasy `AppAplication`, którą musimy zdefiniować jako context - ponieważ rozszerza ona klasę `Application`, czas żecia `ROOM` jest tożsamy z czasem życia samej aplikacji.

In [ ]:
@HiltAndroidApp
public class AppApplication extends Application {}

In [ ]:
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools">

    <application
        android:name=".AppApplication"
...

Dodajmy moduł aplikacji - jak w poprzednim przykładzie, wykorzystamy `SingletonComponent`

In [ ]:
@Module
@InstallIn(SingletonComponent.class)
public class AppModule {

Do modułu dodajmy metodę `getAppDB` dostarczającą instancję bazy danych

In [ ]:
@Singleton
@Provides
AppDatabase getAppDB(Application app){
    return AppDatabase.getDatabase(app);
}

Jak widzimy `getAppDB` przyjmuje instancję `Application` - `Hilt` automatycznie dostarcza instancję klasy oznaczonej jako `@HiltAndroidApp`.

Drugą metodą jest `getDao` dostarczająca instancję obiektu implementującego interfejs `AppDao`.

In [ ]:
@Singleton
@Provides
AppDao getDao(AppDatabase db){
    return db.appDao();
}

Następnym elementem **MVVM** będzie repozytorium, po raz kolejny posłużymy się interfejsem oraz klasą implementującą ten interfejs.

In [ ]:
public interface AppRepository {
    LiveData<List<Student>> readAllData();
    void insert(Student student);
}

In [ ]:
public class AppRepositoryImpl implements AppRepository {

    private final AppDao appDao;

    @Inject
    public AppRepositoryImpl(AppDao appDao) {
        this.appDao = appDao;
    }

    @Override
    public LiveData<List<Student>> readAllData() {
        return appDao.readAllData();
    }

    @Override
    public void insert(Student student) {
        appDao.addItem(student);
    }
}

Do repozytorium dodajemy instancję `AppDao` stosując wstrzyknięcie przez konstruktor.

Powróćmy do `AppModule` i dodajmy metodę dostarczającą repozytorium

In [ ]:
@Singleton
@Provides
AppRepository provideAppRepository(AppDao appDao){
    return new AppRepositoryImpl(appDao);
}

Ostatnim elementem **MVVM** jest `ViewModel`. Przyjmuje dwa argumenty - `Application` oraz `AppRepository` - które dostarczymy stosując wstrzyknięcie przez konstruktor.

In [ ]:
@HiltViewModel
public class AppViewModel extends AndroidViewModel {

    private final AppRepository repository;

    @Inject
    public AppViewModel(Application app, AppRepository repository) {
        super(app);
        this.repository = repository;
    }

Kontekst aplikacji musimy przekazać do konstruktora klasy `AndroidViewModel`, jest to jedyne miejsce gdzie jest on wymagany, więc nie musimy tworzyć pola - stąd brak `val`/`var`.

Dodajmy dwie wcześniej opisane metody.

In [ ]:
LiveData<List<Student>> readAllData(){
    return repository.readAllData();
}

void insert(Student student){
    AppDatabase.databaseWriteExecutor.execute(() -> repository.insert(student));
}

Główną aktywność oznaczamy jako `@AndroidEntryPoint`. Dodajmy `ViewModel`.

In [ ]:
@AndroidEntryPoint
public class MainActivity extends AppCompatActivity {

    private AppViewModel viewModel;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        viewModel = new ViewModelProvider(this).get(AppViewModel.class);


W metodzie `onCreate` dodajmy kilka elementów do bazy oraz obserwator.

In [ ]:
@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);

    TextView textView = findViewById(R.id.textview);

    viewModel = new ViewModelProvider(this).get(AppViewModel.class);

    viewModel.insert(new Student(0, "Rafa"));
    viewModel.insert(new Student(0, "Maciej"));
    viewModel.insert(new Student(0, "Kuba"));

    viewModel.readAllData().observe(this, students ->{
        StringBuilder content = new StringBuilder();
        students.forEach(student -> content
                .append("id: ").append(student.getId()).append("\n")
                .append("Name: ").append(student.getName()).append("\n\n"));
        textView.setText(content.toString());
    });
}